<a href="https://colab.research.google.com/github/Chandan801/Dr.Proton/blob/master/Linear%20reg(bang%20house%20price).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded=files.upload()

Saving Bengaluru_House_Data.csv to Bengaluru_House_Data.csv


# Load Data

In [2]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 

In [3]:
import io
df=pd.read_csv(io.StringIO(uploaded['Bengaluru_House_Data.csv'].decode('utf-8')))
#df_test=pd.read_csv(io.StringIO(uploaded['test - Copy.csv'].decode('utf-8')))
#df_sample=pd.read_csv(io.StringIO(uploaded['sample_submission.csv'].decode('utf-8')))

In [ ]:
df.shape


(13320, 9)

In [4]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [ ]:
df.isnull().sum()

area_type          0
availability       0
location           1
size              16
society         5502
total_sqft         0
bath              73
balcony          609
price              0
dtype: int64

In [5]:
X=df.drop('price',axis=1)
y=df['price']

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.20, random_state=51)

# Preprocessing

In [7]:
from sklearn.impute import SimpleImputer

In [8]:
X_train.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony
11654,Super built-up Area,18-Jun,Javarandoddi,1 BHK,NaN,648,1.0,1.0
4415,Built-up Area,Ready To Move,Koramangala,3 BHK,KHage G,1760,2.0,0.0
10003,Built-up Area,Ready To Move,Thanisandra,3 BHK,Pronyt,1241,2.0,1.0
11573,Super built-up Area,Ready To Move,Whitefield,2 BHK,BrlisCo,1270,2.0,1.0
4160,Built-up Area,Ready To Move,Hormavu,3 BHK,Cadel C,1725,3.0,2.0


In [9]:
num_var=X_train.select_dtypes(include=["int64","float64"]).columns
num_var

Index(['bath', 'balcony'], dtype='object')

In [ ]:
X_train[num_var].isnull().sum()

bath        59
balcony    486
dtype: int64

In [10]:
Imputer_mean=SimpleImputer(strategy='mean')
Imputer_mean.fit(X_train[num_var])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='mean', verbose=0)

In [11]:
X_train[num_var]=Imputer_mean.transform(X_train[num_var])
X_test[num_var]=Imputer_mean.transform(X_test[num_var])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [ ]:
X_train[num_var].isnull().sum()

bath       0
balcony    0
dtype: int64

In [12]:
cat_var=X_train.select_dtypes('object').columns
cat_var

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft'],
      dtype='object')

In [13]:
Imputer_mode=SimpleImputer(strategy='most_frequent')
Imputer_mode.fit(X_train[cat_var])

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [14]:
X_train[cat_var]=Imputer_mode.transform(X_train[cat_var])
X_test[cat_var]=Imputer_mode.transform(X_test[cat_var])
X_train[cat_var].isnull().sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

area_type       0
availability    0
location        0
size            0
society         0
total_sqft      0
dtype: int64

## Label and Ordinal Encoding

In [ ]:
X_train['size']

11654         1 BHK
4415          3 BHK
10003         3 BHK
11573         2 BHK
4160          3 BHK
6328          1 BHK
11303     6 Bedroom
8937          2 BHK
1800          3 BHK
10507         3 BHK
3463          3 BHK
6607          8 BHK
9827          2 BHK
3072          3 BHK
13213     4 Bedroom
7600          4 BHK
4611          2 BHK
8264          3 BHK
2414      2 Bedroom
8301          4 BHK
8946          3 BHK
6676          2 BHK
4945          3 BHK
6578          3 BHK
8817          2 BHK
5925          3 BHK
6168      8 Bedroom
1459      5 Bedroom
12752     6 Bedroom
6487          2 BHK
5665          2 BHK
1333          3 BHK
8718          2 BHK
8733          3 BHK
7543          1 BHK
5445          3 BHK
275           3 BHK
11749         3 BHK
1790          2 BHK
10542     4 Bedroom
10475     9 Bedroom
4382          2 BHK
5308          2 BHK
10753         2 BHK
10646         2 BHK
1968          2 BHK
3503          1 BHK
7312          2 BHK
2931      4 Bedroom
7485          2 BHK


In [16]:
X_train.columns
X_test.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'bath', 'balcony'],
      dtype='object')

In [17]:
X_train2=X_train[['area_type', 'availability', 'location', 'size', 'society','total_sqft']]
X_test2=X_test[['area_type', 'availability', 'location', 'size', 'society','total_sqft']]

In [18]:
X_train2['area_type'].value_counts()
X_test2['area_type'].value_counts()

Super built-up  Area    1750
Built-up  Area           482
Plot  Area               408
Carpet  Area              24
Name: area_type, dtype: int64

In [19]:
order_label1={"Super built-up  Area":1,"Built-up  Area":2,"Plot  Area":3,"Carpet  Area":4}

In [20]:
X_train2['area_type_enc']=X_train2['area_type'].map(order_label1)
X_train2['area_type_enc'].isnull().sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0

In [21]:
X_test2['area_type_enc']=X_test2['area_type'].map(order_label1)
X_test2['area_type_enc'].isnull().sum()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0

In [22]:
X_train2['size'].value_counts()

2 BHK         4197
3 BHK         3445
4 Bedroom      661
4 BHK          463
3 Bedroom      438
1 BHK          432
2 Bedroom      258
5 Bedroom      242
6 Bedroom      148
1 Bedroom       85
8 Bedroom       70
7 Bedroom       62
5 BHK           45
9 Bedroom       33
6 BHK           26
7 BHK           11
1 RK            10
10 Bedroom      10
9 BHK            7
8 BHK            4
11 Bedroom       2
19 BHK           1
12 Bedroom       1
27 BHK           1
14 BHK           1
43 Bedroom       1
18 Bedroom       1
10 BHK           1
Name: size, dtype: int64

In [23]:
X_test2['size'].isnull().sum()

0

In [24]:
from sklearn.preprocessing import LabelEncoder

In [25]:
lE1=LabelEncoder()

lE1.fit_transform(X_train2['size'])

array([ 0, 13, 13, ..., 10, 15, 10])

In [26]:
X_train2['size_enc']=lE1.fit_transform(X_train2['size'])
X_train2.head()

,area_type,availability,location,size,society,total_sqft,area_type_enc,size_enc
11654,Super built-up Area,18-Jun,Javarandoddi,1 BHK,PrarePa,648,1,0
4415,Built-up Area,Ready To Move,Koramangala,3 BHK,KHage G,1760,2,13
10003,Built-up Area,Ready To Move,Thanisandra,3 BHK,Pronyt,1241,2,13
11573,Super built-up Area,Ready To Move,Whitefield,2 BHK,BrlisCo,1270,1,10
4160,Built-up Area,Ready To Move,Hormavu,3 BHK,Cadel C,1725,2,13


In [27]:
lE2=LabelEncoder()

lE2.fit_transform(X_test2['size'])
X_test2['size_enc']=lE2.fit_transform(X_test2['size'])
X_test2.head()

,area_type,availability,location,size,society,total_sqft,area_type_enc,size_enc
11791,Super built-up Area,Ready To Move,GM Palaya,1 BHK,Srg Bnc,675,1,0
3415,Super built-up Area,20-Jun,Jakkur,3 BHK,SoineC,1950,1,10
11516,Super built-up Area,Ready To Move,7th Phase JP Nagar,3 BHK,BriumMi,2100,1,10
3147,Super built-up Area,Ready To Move,Electronics City Phase 1,3 BHK,SNdri N,1515,1,10
3057,Plot Area,Ready To Move,Padmanabhanagar,3 Bedroom,PrarePa,1500,3,11


In [28]:
X_train2['society'].value_counts()
lE2=LabelEncoder()

lE2.fit_transform(X_train2['society'])
X_train2['society_enc']=lE2.fit_transform(X_train2['society'])
X_train2.head()

,area_type,availability,location,size,society,total_sqft,area_type_enc,size_enc,society_enc
11654,Super built-up Area,18-Jun,Javarandoddi,1 BHK,PrarePa,648,1,0,1340
4415,Built-up Area,Ready To Move,Koramangala,3 BHK,KHage G,1760,2,13,873
10003,Built-up Area,Ready To Move,Thanisandra,3 BHK,Pronyt,1241,2,13,1418
11573,Super built-up Area,Ready To Move,Whitefield,2 BHK,BrlisCo,1270,1,10,299
4160,Built-up Area,Ready To Move,Hormavu,3 BHK,Cadel C,1725,2,13,335


In [29]:
X_test2['society'].value_counts()
lE2=LabelEncoder()

lE2.fit_transform(X_test2['society'])
X_test2['society_enc']=lE2.fit_transform(X_test2['society'])
X_test2.head()

,area_type,availability,location,size,society,total_sqft,area_type_enc,size_enc,society_enc
11791,Super built-up Area,Ready To Move,GM Palaya,1 BHK,Srg Bnc,675,1,0,803
3415,Super built-up Area,20-Jun,Jakkur,3 BHK,SoineC,1950,1,10,760
11516,Super built-up Area,Ready To Move,7th Phase JP Nagar,3 BHK,BriumMi,2100,1,10,108
3147,Super built-up Area,Ready To Move,Electronics City Phase 1,3 BHK,SNdri N,1515,1,10,641
3057,Plot Area,Ready To Move,Padmanabhanagar,3 Bedroom,PrarePa,1500,3,11,511


In [30]:
X_train2['location'].value_counts()
lE3=LabelEncoder()

lE3.fit_transform(X_train2['location'])
X_train2['location_enc']=lE3.fit_transform(X_train2['location'])
X_train2.head()



,area_type,availability,location,size,society,total_sqft,area_type_enc,size_enc,society_enc,location_enc
11654,Super built-up Area,18-Jun,Javarandoddi,1 BHK,PrarePa,648,1,0,1340,552
4415,Built-up Area,Ready To Move,Koramangala,3 BHK,KHage G,1760,2,13,873,663
10003,Built-up Area,Ready To Move,Thanisandra,3 BHK,Pronyt,1241,2,13,1418,1056
11573,Super built-up Area,Ready To Move,Whitefield,2 BHK,BrlisCo,1270,1,10,299,1155
4160,Built-up Area,Ready To Move,Hormavu,3 BHK,Cadel C,1725,2,13,335,496


In [31]:
X_test2['location'].value_counts()
lE3=LabelEncoder()

lE3.fit_transform(X_test2['location'])
X_test2['location_enc']=lE3.fit_transform(X_test2['location'])
X_test2.head()

,area_type,availability,location,size,society,total_sqft,area_type_enc,size_enc,society_enc,location_enc
11791,Super built-up Area,Ready To Move,GM Palaya,1 BHK,Srg Bnc,675,1,0,803,225
3415,Super built-up Area,20-Jun,Jakkur,3 BHK,SoineC,1950,1,10,760,303
11516,Super built-up Area,Ready To Move,7th Phase JP Nagar,3 BHK,BriumMi,2100,1,10,108,33
3147,Super built-up Area,Ready To Move,Electronics City Phase 1,3 BHK,SNdri N,1515,1,10,641,221
3057,Plot Area,Ready To Move,Padmanabhanagar,3 Bedroom,PrarePa,1500,3,11,511,483


In [32]:
X_train2['availability'].value_counts()
lE4=LabelEncoder()

lE4.fit_transform(X_train2['availability'])
X_train2['availability_enc']=lE4.fit_transform(X_train2['availability'])
X_train2.head()


,area_type,availability,location,size,society,total_sqft,area_type_enc,size_enc,society_enc,location_enc,availability_enc
11654,Super built-up Area,18-Jun,Javarandoddi,1 BHK,PrarePa,648,1,0,1340,552,31
4415,Built-up Area,Ready To Move,Koramangala,3 BHK,KHage G,1760,2,13,873,663,79
10003,Built-up Area,Ready To Move,Thanisandra,3 BHK,Pronyt,1241,2,13,1418,1056,79
11573,Super built-up Area,Ready To Move,Whitefield,2 BHK,BrlisCo,1270,1,10,299,1155,79
4160,Built-up Area,Ready To Move,Hormavu,3 BHK,Cadel C,1725,2,13,335,496,79


In [33]:
X_test2['availability'].value_counts()
lE4=LabelEncoder()

lE4.fit_transform(X_test2['availability'])
X_test2['availability_enc']=lE4.fit_transform(X_test2['availability'])
X_test2.head()

,area_type,availability,location,size,society,total_sqft,area_type_enc,size_enc,society_enc,location_enc,availability_enc
11791,Super built-up Area,Ready To Move,GM Palaya,1 BHK,Srg Bnc,675,1,0,803,225,58
3415,Super built-up Area,20-Jun,Jakkur,3 BHK,SoineC,1950,1,10,760,303,40
11516,Super built-up Area,Ready To Move,7th Phase JP Nagar,3 BHK,BriumMi,2100,1,10,108,33,58
3147,Super built-up Area,Ready To Move,Electronics City Phase 1,3 BHK,SNdri N,1515,1,10,641,221,58
3057,Plot Area,Ready To Move,Padmanabhanagar,3 Bedroom,PrarePa,1500,3,11,511,483,58


In [34]:
X_train2.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'area_type_enc', 'size_enc', 'society_enc',
       'location_enc', 'availability_enc'],
      dtype='object')

In [35]:
X_train['total_sqft'].value_counts()
lE5=LabelEncoder()

lE5.fit_transform(X_train2['total_sqft'])
X_train2['total_sqft_enc']=lE5.fit_transform(X_train2['total_sqft'])
X_train2.head()

,area_type,availability,location,size,society,total_sqft,area_type_enc,size_enc,society_enc,location_enc,availability_enc,total_sqft_enc
11654,Super built-up Area,18-Jun,Javarandoddi,1 BHK,PrarePa,648,1,0,1340,552,31,1678
4415,Built-up Area,Ready To Move,Koramangala,3 BHK,KHage G,1760,2,13,873,663,79,816
10003,Built-up Area,Ready To Move,Thanisandra,3 BHK,Pronyt,1241,2,13,1418,1056,79,286
11573,Super built-up Area,Ready To Move,Whitefield,2 BHK,BrlisCo,1270,1,10,299,1155,79,320
4160,Built-up Area,Ready To Move,Hormavu,3 BHK,Cadel C,1725,2,13,335,496,79,783


In [36]:
X_test2['total_sqft'].value_counts()
lE5=LabelEncoder()

lE5.fit_transform(X_test2['total_sqft'])
X_test2['total_sqft_enc']=lE5.fit_transform(X_test2['total_sqft'])
X_test2.head()

,area_type,availability,location,size,society,total_sqft,area_type_enc,size_enc,society_enc,location_enc,availability_enc,total_sqft_enc
11791,Super built-up Area,Ready To Move,GM Palaya,1 BHK,Srg Bnc,675,1,0,803,225,58,824
3415,Super built-up Area,20-Jun,Jakkur,3 BHK,SoineC,1950,1,10,760,303,40,534
11516,Super built-up Area,Ready To Move,7th Phase JP Nagar,3 BHK,BriumMi,2100,1,10,108,33,58,566
3147,Super built-up Area,Ready To Move,Electronics City Phase 1,3 BHK,SNdri N,1515,1,10,641,221,58,346
3057,Plot Area,Ready To Move,Padmanabhanagar,3 Bedroom,PrarePa,1500,3,11,511,483,58,341


In [37]:
X_train2.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'area_type_enc', 'size_enc', 'society_enc',
       'location_enc', 'availability_enc', 'total_sqft_enc'],
      dtype='object')

In [38]:
X_test2.columns

Index(['area_type', 'availability', 'location', 'size', 'society',
       'total_sqft', 'area_type_enc', 'size_enc', 'society_enc',
       'location_enc', 'availability_enc', 'total_sqft_enc'],
      dtype='object')

In [39]:
X_test3=X_test2[['area_type_enc', 'size_enc', 'society_enc', 'location_enc',
       'availability_enc','total_sqft_enc']]

In [40]:
X_train3=X_train2[['area_type_enc', 'size_enc', 'society_enc', 'location_enc',
       'availability_enc','total_sqft_enc']]

In [41]:
X_train5=X_train[['bath', 'balcony']]

In [42]:
X_test5=X_test[['bath', 'balcony']]

In [43]:
df_train=pd.concat([X_train5,X_train3],axis=1)
df_train.select_dtypes(include=["int64","float64"]).columns


Index(['bath', 'balcony', 'area_type_enc', 'size_enc', 'society_enc',
       'location_enc', 'availability_enc', 'total_sqft_enc'],
      dtype='object')

In [44]:
df_test=pd.concat([X_test5,X_test3],axis=1)
df_test.select_dtypes(include=["int64","float64"]).columns

Index(['bath', 'balcony', 'area_type_enc', 'size_enc', 'society_enc',
       'location_enc', 'availability_enc', 'total_sqft_enc'],
      dtype='object')

# Feature Scaling

In [45]:
from sklearn.preprocessing import StandardScaler

In [46]:
sc=StandardScaler()
sc.fit(df_train)
df_train=sc.transform(df_train)
df_test=sc.transform(df_test)

# Linear Regresssion

In [47]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()

lr.fit(df_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [48]:
lr.coef_

array([44.12157269,  6.66312248, 10.2947721 , 18.41314628, -1.9078672 ,
        1.90555919, -2.95663889, 26.01200386])

In [49]:
lr.intercept_

112.5253908596096

In [50]:
df_test[0:]

array([[-1.26503342, -0.72844726, -0.65531228, ..., -1.1683294 ,
        -0.65554903,  0.18774608],
       [ 0.23095577,  0.52232239, -0.65531228, ..., -0.93731484,
        -1.62945129, -0.3142274 ],
       [ 0.23095577,  0.52232239, -0.65531228, ..., -1.73698062,
        -0.65554903, -0.25883723],
       ...,
       [-0.51703882,  1.77309205,  0.64773076, ..., -1.50892779,
        -0.65554903, -0.97371671],
       [-0.51703882, -0.72844726,  0.64773076, ..., -1.18906147,
        -3.036199  , -0.98410236],
       [-0.51703882, -0.72844726, -0.65531228, ..., -1.18906147,
        -2.71156492, -1.22643439]])

In [51]:
lr.predict(df_test)

array([ -3.8421274 , 107.25727097, 106.77361542, ...,  66.95881407,
        57.26964905,  36.59175058])

In [52]:
y_test

11791      35.000
3415      131.000
11516     200.000
3147       80.910
3057      200.000
9009       50.000
5186      258.000
868        46.550
9509       41.000
11296      57.320
4359      250.000
12986      45.000
1319       44.000
8388       46.000
9621      295.000
12673      49.820
1169       42.000
11125      39.000
12611     108.000
7060      215.000
7760       47.000
6410       28.000
4795      100.000
10170      80.000
12480     115.000
9689       85.000
10153      49.500
459       360.000
7678      100.000
11104      33.500
8500      111.000
7259       55.000
6585       83.000
5938      400.000
688        83.300
11090     115.000
5855      105.000
5044       25.000
564        28.150
12378      84.500
11244      80.000
8107       35.000
9354       52.320
2846       37.500
1150       41.000
9420      105.000
2605       62.000
1134      115.000
1747       40.000
1700       53.000
8503       34.180
1322       43.850
6271      358.000
1408       69.000
6224       56.000
8558      

In [55]:
lr.score(df_test,y_test)

0.21614363623686195